In [20]:
import json
import requests
import datetime as dt
import pickle

# An api key is emailed to you when you sign up to a plan
api_key = 'da3195c8356b3b0ee3e5a6682ed9c537'


In [26]:
sports_response = requests.get('https://api.the-odds-api.com/v3/odds', params={
    'api_key': api_key,
    'sport': 'basketball_ncaab',
    'region': 'us'
})


# fbs_response = requests.get('https://api.fastbreakstats.com/teams')
# fbs_data = json.loads(fbs_response.text)

sports_json = json.loads(sports_response.text)
# with open('odds.json') as f:
#     sports_json = json.load(f)

In [18]:
def getTeamData(name):
    name = " ".join(name.split()[:-1])
    name = name.replace('-',' ')
    name = name.replace(' ','%20')
    try:
        baseUrl="https://api.fastbreakstats.com/teams/"
        params="?q=current"
        url=baseUrl+name+params
#         print(url)
    
        r = requests.get(url = url).json()[0]
        print(r)
    except IndexError:
        try:
            name = name.replace('%20',' ')
            name = " ".join(name.split()[:-1])
            name = name.replace('-',' ')
            name = name.replace(' ','%20')
            baseUrl="https://api.fastbreakstats.com/teams/"
            params="?q=current"
            url=baseUrl+name+params

            r = requests.get(url = url).json()[0]
            print(r)
        except IndexError:
            print("Team error: " + name)

In [19]:
import numpy as np
# with open('odds.json') as f:
#     sports_json = json.load(f)
    
for game in sports_json['data']:
    for team in game['teams']:
        getTeamData(team)

{'BPIrank': '16', 'BPIval': '9.8', 'KenPomRank': 18, 'KenPomVal': '20.43', 'Opp_Three_Par': '0.444', 'Opp_Tov': '23.3', 'Opp_True_Shoot': '0.461', 'Pace': '74.6', 'SRS': '23.08', 'Team': ['Florida', 'Florida', 'Florida'], 'Three_Par': '0.316', 'Tov': '16.7', 'True_Shoot': '0.6', 'd_eff': '77.53', 'o_eff': '109.48', 'papg': '59.7', 'ppg': '84', 'record_time': 'Sat, 12 Dec 2020 07:00:00 GMT'}
{'BPIrank': '16', 'BPIval': '9.8', 'KenPomRank': 18, 'KenPomVal': '20.43', 'Opp_Three_Par': '0.444', 'Opp_Tov': '23.3', 'Opp_True_Shoot': '0.461', 'Pace': '74.6', 'SRS': '23.08', 'Team': ['Florida', 'Florida', 'Florida'], 'Three_Par': '0.316', 'Tov': '16.7', 'True_Shoot': '0.6', 'd_eff': '77.53', 'o_eff': '109.48', 'papg': '59.7', 'ppg': '84', 'record_time': 'Sat, 12 Dec 2020 07:00:00 GMT'}
{'BPIrank': '60', 'BPIval': '6.1', 'KenPomRank': 32, 'KenPomVal': '17.84', 'Opp_Three_Par': '0.361', 'Opp_Tov': '14.5', 'Opp_True_Shoot': '0.477', 'Pace': '71.7', 'SRS': '10.55', 'Team': ['Kentucky', 'Kentucky', 

KeyboardInterrupt: 

In [ ]:
import math

#Returns the confidence that the home team will win.

def getHomeConf(away, home):
    t1coef_mult = float(away['KenPomVal']) * -0.14904683 + float(away['BPIval']) * 0.02408791 + float(away['SRS']) * -0.00285878
    t2coef_mult = float(home['KenPomVal']) *  0.14617775 + float(home['BPIval']) * -0.05714497+ float(home['SRS']) * 0.03016929
 
    coef_sum = (t1coef_mult + t2coef_mult)+0.53923323
    eTOs = math.exp(coef_sum)
    conf = eTOs / (1+eTOs)
    return(conf)

a = getTeamData('Boise St.')
h = getTeamData('BYU')

print('V1: ' + str(getHomeConf(a, h)))

In [9]:
loaded_model = pickle.load(open('6yr_LR.sav', 'rb'))
loaded_model.coef_

array([[-0.14904683,  0.02408791, -0.00285878,  0.14617775, -0.05714497,
         0.03016929]])

In [10]:
stuff = getTeamData('Kansas')
print(stuff)

{'BPIrank': '144', 'BPIval': '1', 'KenPomRank': 121, 'KenPomVal': '4.56', 'Opp_Three_Par': '0.324', 'Opp_Tov': '28.4', 'Opp_True_Shoot': '0.467', 'Pace': '72.6', 'SRS': '9.83', 'Team': ['Abilene Christian', 'Abilene Christian', 'Abilene Christian'], 'Three_Par': '0.369', 'Tov': '20.3', 'True_Shoot': '0.528', 'd_eff': '83.62', 'o_eff': '100.02', 'papg': '54.5', 'ppg': '69', 'record_time': 'Thu, 10 Dec 2020 07:00:00 GMT'}
None


In [11]:
start_time = sports_json['data'][0]['commence_time']

In [12]:
dt.datetime.fromtimestamp(start_time).strftime("%A, %B %d, %Y %I:%M:%S")

'Friday, December 11, 2020 12:30:00'

In [27]:
for game in sports_json['data']:
    print(game['teams'],game['home_team'])
    for site in game['sites']:
        if site['site_key'] == 'bovada':
            odds = site['odds']['h2h']
            for x in range(len(odds)):
                y = odds[x]
                if y==1:
                    break
                elif y<2:
                    y = round(-100/(y-1))
                else:
                    y = round((y-1)*100)
                odds[x] = y
            print(odds,site['site_key'])
            break
        if site['site_key'] == 'draftkings':
            odds = site['odds']['h2h']
            for x in range(len(odds)):
                y = odds[x]
                if y==1:
                    break
                elif y<2:
                    y = round(-100/(y-1))
                else:
                    y = round((y-1)*100)
                odds[x] = y
            print(odds,site['site_key'])

['Florida Gators', 'Florida St Seminoles'] Florida St Seminoles
[104, -123] draftkings
['Kentucky Wildcats', 'Notre Dame Fighting Irish'] Kentucky Wildcats
[-303, 240] draftkings
['Toledo Rockets', 'UMKC Kangaroos'] Toledo Rockets
[-833, 525] bovada
['IUPUI Jaguars', 'Tennessee St Tigers'] IUPUI Jaguars
[-135, 115] bovada
['South Florida Bulls', 'Wofford Terriers'] South Florida Bulls
[-154, 135] bovada
['Florida Gulf Coast Eagles', 'Miami Hurricanes'] Miami Hurricanes
[1400, -5000] draftkings
['Cincinnati Bearcats', 'Tennessee Volunteers'] Tennessee Volunteers
[255, -333] draftkings
['Boston College Eagles', 'Syracuse Orange'] Boston College Eagles
[140, -161] bovada
['Bryant Bulldogs', 'Stony Brook Seawolves'] Bryant Bulldogs
[-238, 195] draftkings
['Drexel Dragons', 'La Salle Explorers'] Drexel Dragons
[-149, 130] bovada
['Detroit Mercy Titans', 'Western Michigan Broncos'] Western Michigan Broncos
[100, -120] bovada
['TCU Horned Frogs', 'Texas A&M Aggies'] TCU Horned Frogs
[-125, 10